In [ ]:
import numpy
import pandas
import datetime
import time

import nbclient as viz

To make a scatter plot with static data:
<pre>
viz.static( source, type='scatter', **kwargs )
</pre>
First argument is data in JSON format

Here we use a simple convention of labeling variables:
* x: independent variable ( X-axis )
* y: response variable ( Y-axis )
* z: parameter ( legend ) if any

For type 'scatter' X and Y are required arguments, Z is optional.

Y argument can take a list as value. X, Y, Z could be numeric or categorical, as well as ordinal or date-time type of variables.

Other accepted optional arguments:
* title
* colormap
* xformat, yformat, zformat: custom tick / value formatting
* xlabel, ylabel, zlabel: labels for axes and legend
* xlog, ylog: logarithmic scale axis
* xbox, ybox: show background box-plot ( quartiles )
* xlim, ylim: clip display boundaries

The latter might be better achieved with zoom setting bookmark. ( See examples below. )

Custom colormap:
* numeric Z: can be set as a gradient by passing lower and upper color boundaries
* ordinal / categorical Z: can be passed as a list of colors
* categorical Z: can be passed as a dictionary

Lets see some examples:

In [ ]:
df = pandas.DataFrame(numpy.random.randn(100, 3), columns=list('ABC'))
df['B'] = df['B'].apply(lambda b: 1 + 1000*abs(b))
df['C'] = df['C'].apply(lambda c: ['red','blue','yellow','green','purple','orange','lime'][int(abs(round(c)))])
df['D'] = df.index.map(lambda i: datetime.datetime.fromtimestamp(time.time() - 43*i).strftime('%Y-%m-%d %H:%M:%S'))
df['E'] = df['A'].apply(lambda a: 1000*a*a) - df['B']
df.head()

In [ ]:
data = df.to_json(orient='records')

### to keep the index
# data = df.reset_index().to_json(orient='records')

In [ ]:
viz.static(data, type='scatter',
         x='A',
         y='B',
         title='Simple Scatter-Plot Example ',
         height=400, width='100%')

In [ ]:
viz.static(data, type='scatter',
         x='D',
         y='B',
         z='A',
         colormap=['#06f','#f60'],
         title='Scatter-Plot Example ( Continuous Numeric Parameter )',
         xlabel='Timestamp',
         ylabel='Response Variable B',
         zlabel='Parameter A',
         ybox = True,
         height=400, width='100%')

With addditional parameter ( argument Z ) the legend is shown which allows interactive filter/selection of data poins. ( Z-scale is zoomable: the number of bins can be adjusted by mouse-wheeling over. )

In [ ]:
viz.static(data, type='scatter',
         x='A',
         y='B',
         z='C',
         colormap={'red':'red','blue':'blue','purple':'purple','green':'teal','yellow':'orange'},
         title='Scatter-Plot Example ( Categorical Parameter )',
         xlabel='Independent Variable A',
         ylabel='Response Variable B',
         zlabel='Parameter C',
         xbox=True,
         ybox=True,
         height=400, width='100%')

By default the legend will show Z distribution as histogram ( numeric Z ) or pie ( categorical Z ).
This feature is interactive: enables dynamic filtered display of data points.
To remove it pass summary = False argument.

In [ ]:
df = pandas.read_csv('app/static/data/timeseries.csv')
df.head()

In [ ]:
data = df.to_json(orient='records')

If Y argument contains the list, Z is derived from that list, Z argument if passed will be ignored.

In [ ]:
viz.static(data, type='scatter',
         x='date',
         y=['New York','San Francisco','Austin'],
         xlabel='Date',
         ylabel='Temperature (F)',
         zlabel='City',
         xformat='%Y %b %d',
         colormap={'New York':'teal','San Francisco':'purple','Austin':'orange'},
         title='Scatter-Plot Example',
         height=400, width='100%')

In [ ]:
viz.static(data, type='scatter',
         x='New York',
         y='San Francisco',
         z='Austin',
         colormap=['blue','red'],
         title='Scatter-Plot Deep-Link Example',
         height=400, width='100%')

If Z is numeric, the histogram appearing on the legend is zoomable: the number of bins can be changed by mouse-wheeling over. Any adjustment or selection can be bookmarked. ( Below is a 'bookmarked' state of the visualization above. )

In [ ]:
from IPython.display import IFrame

IFrame('%s/static/0ffe31dea4378a6c9919e34f3a82f41d/p748300s18000038595870254s34s975400t61262248350019751178262902124996x50640970618456838902977719349029y48916857840429876197238540' % viz.host(),
        height=400, width='100%')

To make a scatter plot with dynamic data source:
<pre>
viz.dynamic( source, type='scatter', **kwargs )
</pre>
First argument is URL end-point to retrieve the data in JSON format. This will always show current data when accessed. With refresh = N argument passed in it will refresh display in N seconds. ( The data is not necessarily time-series type, no persistent connection. )

In [ ]:
viz.dynamic('%s/sample' % viz.host(), type='scatter',
         x='B',
         y='E',
         z='A',
         colormap=['#06f','#f60'],
         title='Scatter-Plot Example ( Dynamic Data Source )',
         xlabel='Variable B',
         ylabel='Variable E',
         zlabel='Parameter A',
         xlim = [0,3000],
         ylim = [-3000,6000],
         zlim = [-2.5, 2.5],
         xbox = True,
         ybox = True,
         refresh=3, ### request for current data in 3 seconds ###
         height=400, width='100%')

With dynamic data source there's no bookmarking capability, but, a png snapshot can be taken and saved locally.

Finally, to make a scatter plot streaming dynamic data source:
<pre>
viz.stream( channel, type='scatter', **kwargs )
</pre>
First argument is URL end-point emitting data. This will refresh display as new data arrives.
With argument stream = N passed in it will keep sliding time-window of size N seconds in display.
( Time-series type of data with persistent connection. )
The data points appear on the plot as they arrive and disappear when older than the time-window specified.

In [ ]:
viz.stream('sample-io', type='scatter',
         x='B',
         y='E',
         z='C',
         colormap={'red':'red','blue':'blue','purple':'purple','yellow':'orange','green':'teal'},
         title='Scatter-Plot Example ( Streaming Data )',
         xlabel='Variable B',
         ylabel='Variable E',
         zlabel='Parameter C',
         xlim = [0,3000],
         ylim = [-3000,6000],
         zdomain = ['red','blue','yellow','green','purple'],
         window = 120, ### 2 minute sliding time window in display ###
         height=400, width='100%')

In [ ]:
viz.stream('sample-io', type='scatter',
         x='B',
         y='E',
         z='A',
         colormap=['#06f','#f60'],
         title='Scatter-Plot Example ( Streaming Data )',
         xlabel='Variable B',
         ylabel='Variable E',
         zlabel='Parameter A',
         xlim = [0,3000],
         ylim = [-3000,6000],
         zlim = [-2.5, 2.5],
         window = 60, ### 1 minute sliding time window in display ###
         height=400, width='100%')